# The Data object and PSF homogenization

In this notebook, we will have a first look at the `galfind.Data` object. The `Data` object stores all the information pertaining to the photometric imaging in a specific survey, including the paths/extensions to the SCI/WHT/ERR maps for each band as well as a `galfind.Instrument` object which includes the relevant filter information.

We will begin by first loading the reduced JADES Origins Field (JOF) JWST/NIRCam imaging from Adams et al. 2024. This data must first be loaded from the EPOCHS dropbox, instructions for which can be found in [Getting started / Downloading observational data](../getting_started/downloading_observational_data.rst). If these data products are not currently downloaded and stored in the relevant locations, please take the time to do this now before continuuing.

To start off, we will practice using the `Data.__init__()` instantiator to load in all the required data. To do this we require the paths/extensions to each of the SCI/WHT/RMS_ERR images, which we will populate in a dictionary.

In [ ]:
# imports
from galfind import config, Data, NIRCam

survey = "JOF"
version = "v11"

# Array storing all NIRCam bands used in the pipeline for JOF
JOF_nircam_bands = ["F115W", "F150W", "F162M", "F182M", "F200W", "F210M", "F250M", "F277W", "F335M", "F356W", "F410M"] # "F090W", "F444W", here too
alignment_band = "F356W"
# Create an instrument object for NIRCam incorporating all the NIRCam bands
NIRCam_instrument = NIRCam(excl_bands = [band for band in NIRCam().band_names if band not in JOF_nircam_bands])

# Proposal IDs for the NIRCam bands
PIDs = {band: "1210" if band.endswith("W") or band == "F410M" else "3215" for band in JOF_nircam_bands}
# Paths to the sci images for the NIRCam bands
im_dir = f"{config['DEFAULT']['GALFIND_DATA']}/jwst/JOF/NIRCam/mosaic_1084_wispnathan"
sci_paths = {band: f"{im_dir}/jw0{PIDs[band]}-o001_t002_nircam_clear-{band.lower()}_i2dnobg.fits" for band in JOF_nircam_bands}
# Paths to the rms error/weight maps are the same as the sci images, just with a different extension
rms_err_paths = sci_paths
wht_paths = sci_paths

# sci, wht, and rms_err extensions for the NIRCam bands
sci_exts = {band: 1 for band in JOF_nircam_bands}
rms_err_exts = {band: 2 for band in JOF_nircam_bands}
wht_exts = {band: 4 for band in JOF_nircam_bands}

data = Data(survey, version, NIRCam_instrument, sci_paths, sci_exts, rms_err_paths, rms_err_exts, wht_paths, wht_exts, alignment_band = alignment_band)

# Print useful information about the data object
print(data)

As you can see from the length of the above section of code, this is a little bit of a handful. In addition, we also require prior knowledge of the available bands for each field as well as the paths/extensions to each SCI/ERR/WHT map. Luckily there is a useful class method to instantiate the `Data` class, `Data.from_survey_version`, which simplifies things for us a bit. We demonstrate its use below.

In [ ]:
# create a Data object directly from the given survey and version
data_from_survey_version = Data.from_survey_version(survey, version)

# ensure that this object is the same as the one created above
assert data == data_from_survey_version